In [1]:
import os
import sys
import numpy as np
import pandas as pd
import collections
from mirnylib import h5dict, genome, plotting
from hiclib import mapping, fragmentHiC, highResBinnedData

hello from new mapping


In [2]:
genome_db = genome.Genome('/home/xli/cloud_research/PengGroup/ZZeng/Data/Haihui/Tcf1/HiC-seq/Jun2016/exp/iterative_mapping/genome.fa')

In [3]:
fname_list = ['WT_CD8_p%d_mapped_reads.hdf5' % i for i in range(1,7)]

mapped_reads_list = []
for fname in fname_list:
    mapped_reads_list.append(h5dict.h5dict(fname))

combine_mapped_reads = h5dict.h5dict('WT_CD8_mapped_reads.hdf5')

combine_mapped_reads['chrms1'] = np.hstack(map(lambda k: k['chrms1'], mapped_reads_list))
combine_mapped_reads['chrms2'] = np.hstack(map(lambda k: k['chrms2'], mapped_reads_list))
combine_mapped_reads['cuts1'] = np.hstack(map(lambda k: k['cuts1'], mapped_reads_list))
combine_mapped_reads['cuts2'] = np.hstack(map(lambda k: k['cuts2'], mapped_reads_list))
combine_mapped_reads['misc'] = map(lambda k: k['misc'], mapped_reads_list)[0]
combine_mapped_reads['strands1'] = np.hstack(map(lambda k: k['strands1'], mapped_reads_list))
combine_mapped_reads['strands2'] = np.hstack(map(lambda k: k['strands1'], mapped_reads_list))


In [4]:
fragments = fragmentHiC.HiCdataset(
    filename='WT_CD8_fragment_dataset.hdf5',
    enzymeName='HindIII',
    genome=genome_db,
    mode='w')

----> New dataset opened, genome iterative_mapping, filename = WT_CD8_fragment_dataset.hdf5


In [5]:
fragments.parseInputData(
    dictLike='WT_CD8_mapped_reads.hdf5', keepSingleSided=False, keepSameFragment=True, keepReadsMolecules=True)

     loading data from file WT_CD8_mapped_reads.hdf5 (assuming h5dict)
raw read count file {0} does not exist
Count not find raw read count
filtering SS reads
Keeping same-fragment reads
keeping --> <-- reads even if they exceed maximumMoleculeLength
300_validPairs:  404008692


In [6]:
fragments.filterDuplicates(mode='ram')
fragments._sortData()

fragments = fragmentHiC.HiCdataset('WT_CD8_fragment_dataset.hdf5', genome_db, enzymeName='HindIII')

fragments.saveByChromosomeHeatmap('WT_CD8_heatmap-res-10K.hdf5', resolution=10000)

fragments.printMetadata(saveTo="WT_CD8_statistics.txt")

----->Filtering duplicates in DS reads: 
Sorting using 2 chunks
----> New dataset opened, genome iterative_mapping, filename = WT_CD8_fragment_dataset.hdf5
----->!!!File already exists! It will be opened in the append mode



/opt/miniconda2/lib/python2.7/site-packages/hiclib/fragmentHiC.py:600: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if outVariable[1] == "ToDefine":
/opt/miniconda2/lib/python2.7/site-packages/hiclib/fragmentHiC.py:600: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  if outVariable[1] == "ToDefine":


----> By chromosome Heatmap saved to 'WT_CD8_heatmap-res-10K.hdf5' at 10000 resolution


In [7]:
WT_CD8_BD = highResBinnedData.HiResHiC(genome_db, 10000, 'WT_CD8_heatmap-res-10K.hdf5', mode='r+')
WT_CD8_BD.loadData('WT_CD8_heatmap-res-10K.hdf5')
outdir = '/home/xli/cloud_research/PengGroup/ZZeng/Data/Haihui/Tcf1/HiC-seq/Combine_Jun_Jul_2016/iterative_mapping_3/3d_genome_browser_WT_CD8/'
pd.DataFrame({'chrmLabels': WT_CD8_BD.genome.chrmLabels, 'chrmLens': WT_CD8_BD.genome.chrmLens}).to_csv(outdir + 'mm9_genome_size.txt', index=None, header=None, sep='\t')
pd.DataFrame({'chrmLabels': WT_CD8_BD.genome.chrmLabels, 'filename': map(lambda chrm: outdir + 'WT_CD8_' + chrm + '.matrix', WT_CD8_BD.genome.chrmLabels)}).to_csv(outdir + 'WT_CD8_intrachrom_matrix_list.txt', index=None, header=None, sep='\t')


In [8]:
for index, chrmLabel in WT_CD8_BD.genome.idx2label.iteritems():
    key = '(%d, %d)' % (index, index)
    print key
    outputfile = 'WT_CD8_' +  chrmLabel + '.matrix'
    
    
    np.savetxt(outdir + outputfile, WT_CD8_BD._h5dict[key], fmt='%.2e', delimiter='\t')

(0, 0)
(1, 1)
(2, 2)
(3, 3)
(4, 4)
(5, 5)
(6, 6)
(7, 7)
(8, 8)
(9, 9)
(10, 10)
(11, 11)
(12, 12)
(13, 13)
(14, 14)
(15, 15)
(16, 16)
(17, 17)
(18, 18)
(19, 19)
(20, 20)
(21, 21)


In [ ]:
BD = highResBinnedData.HiResHiC(genome_db, 10000, 'WT_CD8_heatmap-res-10K-IC.hdf5')
BD.loadData('WT_CD8_heatmap-res-10K.hdf5')

In [ ]:
BD.removeDiagonal()

BD.iterativeCorrection()

## Genome Size File

In [4]:
outdir = '/home/lxiang/cloud_research/PengGroup/ZZeng/Data/Haihui/Tcf1/HiC-seq/Combine_Jun_Jul_2016/iterative_mapping_3/3d_genome_browser_test/'

In [12]:
pd.DataFrame({'chrmLabels': WT_CD8_BD.genome.chrmLabels, 'chrmLens': WT_CD8_BD.genome.chrmLens}).to_csv(outdir + 'mm9_genome_size.txt', index=None, header=None, sep='\t')

In [27]:
pd.DataFrame({'chrmLabels': WT_CD8_BD.genome.chrmLabels, 'filename': map(lambda chrm: outdir + 'WT_CD8_' + chrm + '.matrix', WT_CD8_BD.genome.chrmLabels)}).to_csv(outdir + 'WT_CD8_intrachrom_matrix_list.txt', index=None, header=None, sep='\t')

In [28]:
for index, chrmLabel in WT_CD8_BD.genome.idx2label.iteritems():
    key = '(%d, %d)' % (index, index)
    outputfile = 'WT_CD8_' +  chrmLabel + '.matrix'
    np.savetxt('3d_genome_browser/' + outputfile, WT_CD8_BD._h5dict[key], fmt='%.2e', delimiter='\t')